In [3]:
import numpy as np
import random
print("Sucessfully impoerting NumPy and random libraries")

Sucessfully impoerting NumPy and random libraries


In [4]:
print(random.choice([0,1,2]))

2


In [38]:
states = np.arange(5)
actions = ['GREEN','RED']
print(states)
print(actions)

[0 1 2 3 4]
['GREEN', 'RED']


In [39]:
Q =np.zeros((len(states),len(actions)))
alpha =0.1 # Learning rate
gama = 0.9 # Discount factor
epsilon = 0.2 # Exploration rate
episodes = 300 # Training runs

In [40]:
def get_reward(states,action):
    if 3 <= states < 5:  #heavy 3,4
        if action == 'GREEN':
            reward = 10 # clears congestion
        else:
            reward = -10 #create jams
    elif states == 0:  # empty road
        if action == 'RED':
            reward =  5 # saves energy
        else:
            reward = -5 # wastes power
    else:
        reward = 1 # moderate
    return reward

print('Reward Example (level=3, red):', get_reward(3, 'RED'))

Reward Example (level=3, red): -10


In [41]:
def next_traffic(current,action):
    if action == 'GREEN':
        current += np.random.choice([-1, 0, 1])
    else:
        current -= np.random.choice([-1, 0, 1])
    return int(np.clip(current, 0, 4) )

current = np.random.randint(0, 5)
print(f"Current traffic: {current} → Next traffic: {next_traffic(current,'GREEN')}")

Current traffic: 1 → Next traffic: 0


In [42]:
for ep in range(episodes):
    traffic = random.choice(states)
    for _ in range(15): # steps per episode
        if random.uniform(0, 1) < epsilon:
            action = random.choice(actions)
        else:
            action = actions[np.argmax(Q[traffic])]
 
        next_state = next_traffic(traffic, action)
        reward = get_reward(next_state, action)
 
        a = actions.index(action)
        best_next = np.max(Q[next_state])
        Q[traffic, a] += alpha * (reward + gama * best_next - Q[traffic, a])
print("Training Completed")

Training Completed


In [45]:
try:
    state = int(input('Enter starting traffic state (0–4): '))
    if state < 0 or state > 4:
        raise ValueError('Traffic level out of range!')
except ValueError as e:
    print(e)
    state = 2
    print('Defaut level set to 2')

print(f'\nStarting with state: {state}')
print('Simulating for 10 steps:\n')


for step in range(10):
    action = actions[np.argmax(Q[state])]
    print(f'Step {step+1}: State={state} -> Action={action}')
    state= next_traffic(state, action)

print('\nSimulation complete. Traffic Control finished.')

Enter starting traffic state (0–4):  1



Starting with state: 1
Simulating for 10 steps:

Step 1: State=1 -> Action=RED
Step 2: State=0 -> Action=RED
Step 3: State=0 -> Action=RED
Step 4: State=0 -> Action=RED
Step 5: State=1 -> Action=RED
Step 6: State=2 -> Action=GREEN
Step 7: State=1 -> Action=RED
Step 8: State=2 -> Action=GREEN
Step 9: State=1 -> Action=RED
Step 10: State=1 -> Action=RED

Simulation complete. Traffic Control finished.


In [11]:
print("=== Analysis ===")
print("- Does the system keep GREEN for heavy traffic?")
heavy_preferred = "GREEN" if Q[3,0] > Q[3,1] and Q[4,0] > Q[4,1] else "not consistently GREEN"
print(f"  * For heavy/very heavy (states 3 & 4), policy is {heavy_preferred}.")
print(f"  * Q-values for heavy states: state 3 -> {np.round(Q[3],3)}, state 4 -> {np.round(Q[4],3)}")

print("\n- Does it turn RED when the road is empty?")
empty_preferred = "RED" if Q[0,1] > Q[0,0] else "not consistently RED"
print(f"  * For empty (state 0), policy is {empty_preferred}.")
print(f"  * Q-values for empty state: {np.round(Q[0],3)}")

print("\n- What happens when epsilon is increased or decreased?")
print("  * Increasing epsilon -> more exploration, Q-table learns more varied outcomes but converges slower.")
print("  * Decreasing epsilon -> more exploitation, policy stabilizes faster but might get stuck in suboptimal behavior.")

print("\n- How does Q-table change over time?")
print("  * Early training: small values and noisy preferences.")
print("  * Later: Q-values for good actions (GREEN in heavy, RED in empty) grow larger; bad actions get lower values.")
print("\n(You can re-run training with different hyperparameters to observe changes.)")


=== Analysis ===
- Does the system keep GREEN for heavy traffic?
  * For heavy/very heavy (states 3 & 4), policy is not consistently GREEN.
  * Q-values for heavy states: state 3 -> [20.281 24.725], state 4 -> [20.556 23.899]

- Does it turn RED when the road is empty?
  * For empty (state 0), policy is RED.
  * Q-values for empty state: [19.758 24.455]

- What happens when epsilon is increased or decreased?
  * Increasing epsilon -> more exploration, Q-table learns more varied outcomes but converges slower.
  * Decreasing epsilon -> more exploitation, policy stabilizes faster but might get stuck in suboptimal behavior.

- How does Q-table change over time?
  * Early training: small values and noisy preferences.
  * Later: Q-values for good actions (GREEN in heavy, RED in empty) grow larger; bad actions get lower values.

(You can re-run training with different hyperparameters to observe changes.)
